In [82]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import re
from sklearn.model_selection import train_test_split


In [83]:
tokenizer = AutoTokenizer.from_pretrained("ulrichING/bert-base-sentiment-analysis-french")
model = AutoModelForSequenceClassification.from_pretrained("ulrichING/bert-base-sentiment-analysis-french")

In [84]:
def tweet_test(logits):
    rep = logits.tolist()
    rep = rep[0]
    indice_max = rep.index(max(rep)) + 1
    if indice_max == 1 :
        return 0
    else :
        return 4

In [85]:

column_names = ['class', 'id', 'date', 'query', 'name', 'tweet']
df = pd.read_csv("dataSet140.csv", sep=',', encoding='ISO-8859-1',names=column_names)


In [86]:
def cleanText(text):
    clean_text = re.sub(r'@\w+|http\S+', '', text)
    return clean_text.strip()


In [87]:
def test(df):
    count = 0
    df_cleaned = df.loc[:, "tweet_fr"] = df["tweet_fr"].apply(cleanText)
    ans = []
    for item in df_cleaned:
        text = item
        inputs = tokenizer(text, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        predicted_class = torch.argmax(logits).item()

        print(text)
        print(f"Raw logits: {logits}")
        print(f"Predicted sentiment class: {predicted_class + 1} (1 = worst, 2 = best)")
        count+=1
        print(count)
        ans.append(tweet_test(logits))
    return ans

In [88]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

print("Training set shape:", train_df.shape)
print("Test set shape:", test_df.shape)


Training set shape: (1120000, 6)
Test set shape: (480000, 6)


In [89]:
test_df = pd.read_csv("sentiment140_fr_2000.csv")


In [90]:
test_df.head()

,class,id,date,query,name,tweet,tweet_fr
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"- Awww, that's a bummer. You shoulda got Davi...",Vous devriez avoir David Carr du troisième jou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,est contrarié qu'il ne puisse pas mettre à jou...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,I dived many times for the ball. Managed to sa...,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,Tout mon corps se sent démangeant et comme sur...
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"no, it's not behaving at all. i'm mad. why am ...","Non, ça ne se passe pas du tout. Je suis en co..."


In [91]:
'''
test_df.to_csv("test_df.csv", index=False)
train_df.to_csv("train_df.csv", index=False)
'''

'\ntest_df.to_csv("test_df.csv", index=False)\ntrain_df.to_csv("train_df.csv", index=False)\n'

In [92]:
test_df.loc[:, "tweet_fr"] = test_df["tweet_fr"].apply(cleanText)


In [93]:
ans = test(test_df[:2000])

Vous devriez avoir David Carr du troisième jour pour le faire. ;D
Raw logits: tensor([[ 1.0067, -1.2613]])
Predicted sentiment class: 1 (1 = worst, 2 = best)
1
est contrarié qu'il ne puisse pas mettre à jour son Facebook en le texto... et pourrait pleurer en conséquence École aujourd'hui aussi.
Raw logits: tensor([[ 0.7369, -0.9678]])
Predicted sentiment class: 1 (1 = worst, 2 = best)
2
J'ai plongé plusieurs fois pour la balle. A réussi à économiser 50% Le reste sort des limites
Raw logits: tensor([[ 1.3650, -1.5849]])
Predicted sentiment class: 1 (1 = worst, 2 = best)
3
Tout mon corps se sent démangeant et comme sur le feu
Raw logits: tensor([[-0.4242,  0.2913]])
Predicted sentiment class: 2 (1 = worst, 2 = best)
4
Non, ça ne se passe pas du tout. Je suis en colère. Pourquoi suis-je ici ? Parce que je ne peux pas te voir partout.
Raw logits: tensor([[ 2.1246, -2.2530]])
Predicted sentiment class: 1 (1 = worst, 2 = best)
5
pas toute l'équipage
Raw logits: tensor([[ 1.0247, -1.4003]])
P

In [94]:
ans1 = 0
for actual, predicted in zip(test_df["class"], ans):
    if actual == predicted:
        ans1 += 1

accuracy = ans1 / test_df[:2000].shape[0]
print(accuracy)


0.95
